# 2. Uproot

<br><br><br><br><br>

## What a ~complete analysis looks like in Uproot/Awkward Array

Instead of starting with small steps, let's look at where this is going, what a sample analysis looks like with these tools.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak

import uproot
import hist

In [ ]:
upfile = uproot.open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
uptree = upfile["Events"]
uptree.show()

The general strategy is to get arrays in one function call (usually slow, has to read) and use them interactively afterward.

In [ ]:
muons = uptree.arrays(["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"], cut="nMuon >= 2", how="zip", entry_stop=100000)

We've already applied an `nMuon >= 2` cut, but we can define additional cuts.

In [ ]:
os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
os_cut

Slicing (to be described in more detail later) can remove data and reduce the structure of an array.

In [ ]:
mu1 = muons[os_cut, 0, "Muon"]
mu2 = muons[os_cut, 1, "Muon"]
mu1, mu2

Make a histogram and fill it with a calculation from the array. The mini-plot is just the way this histogram type is visualized in Jupyter.

In [ ]:
h1 = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()

In [ ]:
h1.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

Plot it using Matplotlib (for logscale).

In [ ]:
h1.plot()
plt.yscale("log")

<br><br><br><br><br>

## What a the same analysis looks like in PyROOT

In [ ]:
import ROOT
c1 = ROOT.TCanvas()

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

ROOT analyses (before RDataFrame; see below) are based on an event loop. Reading and calculations are done in the loop.

This is not following the "one weird trick." That's why it's slow.

In [ ]:
h2 = ROOT.TH1D("h2", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # Analyzing a subsample means breaking out of the loop early.
    if index == 100000:
        break
    # Applying cuts means if-statements.
    if event.nMuon >= 2 and event.Muon_charge[0] != event.Muon_charge[1]:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]
        h2.Fill(np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi))))

In [ ]:
h2.Draw()
c1.SetLogy()
c1.Draw()

<br><br><br><br><br>

## What a the same analysis looks like in old C++

By "old C++," I mean `TTree::GetEntry`. This is also a reading + calculating loop over events.

Use `ROOT.gInterpreter.Declare` to define a C++ function in Python that we can use through PyROOT.

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& h3, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);
    
    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] != Muon_charge[1]) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            h3.Fill(sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi))));
        }
    }
}
''')

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

h3 = ROOT.TH1D("h3", "mass", 120, 0, 120)

ROOT.compute(h3, roottree)

In [ ]:
h3.Draw()
c1.SetLogy()
c1.Draw()

<br><br><br><br><br>

## What a the same analysis looks like in modern RDataFrame

This case mixes Python (for organization) with C++ (for speed).

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_limit = df.Range(100000)
df_2mu = df_limit.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

# This one is an endpoint (action).
h4 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

The above just sets up the calculation (compiling the C++ strings). It runs when you evaluate `h4.Draw`.

In [ ]:
h4.Draw()   # <--- This is the line that computes everything.
c1.SetLogy()
c1.Draw()

For more on RDataFrame, see [this tutorial](https://cms-opendata-workshop.github.io/workshop-lesson-root/05-rdataframe/index.html).

<br><br><br><br><br>

## Ways to get data from Uproot

Uproot provides a rather low-level view into a ROOT file, so let's start with terminology.

All of the links below go to [the documentation](https://uproot.readthedocs.io/en/latest/).

<img src="img/terminology.svg" style="width: 800px">

<br><br><br>

### Navigating TDirectories

When you [open](https://uproot.readthedocs.io/en/latest/uproot.reading.open.html) a [TFile](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyFile.html) in Uproot, you actually get a [TDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html) object.

In [ ]:
import numpy as np
import awkward as ak
import uproot

In [ ]:
directory = uproot.open("data/nesteddirs.root")
directory

That's because it's the [TDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html) that shows you all the objects that could be read.

You'll rarely need it, but the [TFile](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyFile.html) itself is accessible from every object.

In [ ]:
file = directory.file
file

In [ ]:
file.file_path

In [ ]:
file.root_version

In [ ]:
file.uuid

The [TDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html) acts like a Python [Mapping](https://docs.python.org/3/library/collections.abc.html#collections.abc.Mapping), meaning that it has [keys](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#keys), [values](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#values), and [items](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#items), and you can get any value with `directory[key_name]`.

In [ ]:
directory.keys()

In [ ]:
directory["one"]

In [ ]:
directory["one/two/tree"]

In [ ]:
directory["one"]["two"]["tree"]

In [ ]:
directory.values()

In [ ]:
directory.items()

Since you'll likely want to find objects by class name without reading them, there's a fourth method: [classnames](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#classnames).

In [ ]:
directory.classnames()

See the documentation; there are ways to filter the output. You might need that if you have a file with a lot of histograms in it.

In [ ]:
directory.classnames(recursive=False)

In [ ]:
directory.keys(filter_classname="TT*")

<br><br><br>

### Generic objects

ROOT (probably) has thousands of classes. Uproot does not have specialized code to recognize them all.

However, most objects are readable anyway thanks to the [TStreamerInfo](https://uproot.readthedocs.io/en/latest/uproot.streamers.Model_TStreamerInfo.html) in every ROOT file. Here's one with custom classes that Uproot couldn't possibly know about.

In [ ]:
directory = uproot.open("data/icecube-supernovae.root")
directory.classnames()

The classes `SN_Analysis_Configuration_t`, `I3Eval_t`, `SN_File_t` were generated from the [TStreamerInfo](https://uproot.readthedocs.io/en/latest/uproot.streamers.Model_TStreamerInfo.html).

In [ ]:
directory.streamer_of("config/detector")

In [ ]:
directory.file.show_streamers("I3Eval_t")

You can read these objects, but they have no specialized methods and all members have to be accessed through [has_member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#has-member)/[member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#member)/[all_members](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#all-members).

In [ ]:
directory["config/detector"]

In [ ]:
directory["config/detector"].all_members

In [ ]:
directory["config/detector"].member("ChannelIDMap")

If a class has "Unknown" in its name or `isinstance(obj, (uproot.model.UnknownClass, uproot.model.UnknownClassVersion)`, that means that it could not be read.

(I don't know of any examples of that at the moment.)

<br><br><br>

### Histograms and graphs

Other classes have specialized interfaces, like histograms and some graphs. You can view the [axis](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html#axis) [edges](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TAxis.TAxis.html#edges) and the [values](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html#values), but this interface is minimal.

Normally, you'd convert

   * [to_numpy](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html#to-numpy): tuple of arrays (values and edges)
   * [to_boost](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html#to-boost): `boost_histogram` object
   * [to_hist](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html#to-hist): `hist` object (more fully featured subclass of `boost_histogram`)

In [ ]:
directory = uproot.open("data/hepdata-example.root")
directory.classnames()

In [ ]:
directory["hpx"].to_numpy()

In [ ]:
directory["hpx"].to_hist()

In [ ]:
directory["hpxpy"].to_hist()

In [ ]:
directory["hprof"].to_hist()

<br><br><br>

### TTrees

That's what you're here for, most likely.

In [ ]:
directory = uproot.open("data/Zmumu.root")
directory.classnames()

In [ ]:
events = directory["events"]
events.show()

I've been advertising the above as a way to get a first look at a TTree, but you know you can get more detail through [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#keys), [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#typenames), and [TBranch](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html) [interpretation](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#interpretation).

In [ ]:
events.keys()

In [ ]:
events.keys(filter_typename="/int.*/")

In [ ]:
events.typenames()

In [ ]:
{name: branch.interpretation for name, branch in events.items()}

<br><br><br>

### Get one array from one TBranch

This is the simplest method: [array](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#array) (singular).

In [ ]:
events["M"].array()

Some important parameters:

   * `entry_start`, `entry_stop` to limit how much you read (if it's big)
   * `library="np"` for NumPy arrays, `library="ak"` for Awkward Arrays, and `library="pd"` for Pandas (Series or DataFrame)

In [ ]:
events["M"].array(entry_stop=5)

In [ ]:
events["M"].array(library="np")

In [ ]:
events["M"].array(library="ak")   # default

In [ ]:
events["M"].array(library="pd")

<br><br><br>

### Get several TBranches as one array/array group

[TTrees](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) and [TBranches](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html) with subbranches have an [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays) (plural) method.

In [ ]:
events.arrays()

(Don't do that if your file is large.)

The return type is an "array group," which has a different meaning for each of the backends: NumPy, Awkward Array, and Pandas.

In [ ]:
events.arrays(library="np")

In [ ]:
events.arrays(library="ak")

In [ ]:
events.arrays(library="pd")

The first argument of this method takes a list of _expressions_, which could just be branch names:

In [ ]:
events.arrays(["px1", "py1", "px2", "py2"], library="pd")

Or it can be computed expressions.

In [ ]:
events.arrays(["sqrt(px1**2 + py1**2)", "sqrt(px2**2 + py2**2)"], library="pd")

This is to support any [aliases](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#aliases) that might be in the [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html), but you can make up your own `aliases` on the spot.

In [ ]:
events.arrays(["pt1", "pt2"], {"pt1": "sqrt(px1**2 + py1**2)", "pt2": "sqrt(px2**2 + py2**2)"}, library="pd")

The `expressions` parameter is not a good way to select branches by name.

   * nested branches, paths with "`/`", _would be interpreted as division!_
   * wildcards, paths with "`*`", _would be interpreted as multiplication!_

To select branches by name, use `filter_name`, `filter_typename`, `filter_branch` (all in the [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays) documentation).

In [ ]:
events.arrays(filter_name="p[xyz]*", library="pd")

(These filters have the same meaning as in [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#keys) and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#typenames), but those methods do not read potentially large datasets.)

In [ ]:
events.keys(filter_name="p[xyz]*")

In [ ]:
events.typenames(filter_name="p[xyz]*")

<br><br><br>

### Get arrays in manageable chunks

The [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#iterate) method is like [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays), but it can be used in a loop over chunks of the array.

How large are the chunks? You should set that with `step_size`.

In [ ]:
for arrays in events.iterate(step_size=300):
    print(repr(arrays))

In [ ]:
for arrays in events.iterate(step_size="50 kB"):   # 50 kB is very small! for illustrative purposes only!
    print(repr(arrays))

<br><br><br>

### Collections of files (like TChain)

If you want to read a bunch of files in one call, it has to be a function, rather than a method of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html).

   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) is [uproot.concatenate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.concatenate.html). _(Reads everything at once: use this as a convenience on datasets you know are small!)_
   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate) is [uproot.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html). _(This is the most useful one.)_
   * There's also an [uproot.lazy](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.lazy.html) _(More on this below.)_

In [ ]:
import IPython
import matplotlib.pyplot as plt
import matplotlib.pylab

import hist

In [ ]:
h1 = hist.Hist.new.Reg(500, 0, 500, name="mass").Double()
drawables = []

for muons in uproot.iterate(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],

    # branches to read as expressions
    ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"],

    # cut to apply to expressions
    cut="nMuon >= 2",

    # library dependent; for library="ak", try to split branch names at underscore and make nested records (poor man's NanoEvents)
    how="zip",

    # the all-important step_size!
    step_size="1 MB",

    # options you would normally pass to uproot.open
    xrootd_handler=uproot.MultithreadedXRootDSource,
    num_workers=10,
):
    # do everything you're going to do to this array
    os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
    mu1 = muons[os_cut, 0, "Muon"]
    mu2 = muons[os_cut, 1, "Muon"]

    # such as filling a histogram
    h1.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

    # a little magic to animate it; hardest part is removing the previous plots
    for drawable in drawables:
        drawable.stairs.remove()
        drawable.errorbar.remove()
        drawable.legend_artist.remove()
    drawables = h1.plot(color="blue")
    plt.yscale("log")
    IPython.display.display(matplotlib.pylab.gcf())
    IPython.display.clear_output(wait=True)

<br><br><br>

### Lazy arrays

Lazy arrays were introduced so that you can explore a large dataset without knowing ahead of time what parts you're going to read.

In [ ]:
events = uproot.lazy(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],
    # step_size is still important
    step_size="1 MB",
)
events

In [ ]:
events.type

In [ ]:
len(events)

It _looks like_ we've read that big remote file.

But actually (if we peek in the lazy array's cache), we've only read 3 TBaskets.

In [ ]:
events._caches[0].keys()

In [ ]:
events.Muon_pt

Now 4 TBaskets...

In [ ]:
events._caches[0].keys()

<div style="font-size: 25px; margin: 10px; padding: 30px; border: 1px dashed black;">
    Big important warning: <b>this access pattern is not read or memory efficient!</b>
</div>

This pattern is much better for exploration, to compute quantities without having to decide ahead of time which branches you'll need.

**Recommendation:** develop with lazy arrays, but then put the calculation into an [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html) loop or a [Coffea Processor](https://coffeateam.github.io/coffea/notebooks/processor.html).